### Import Library

In [1]:
import base64
import json
import random
import pandas as pd
import requests
import tensorflow as tf

### Load Dataset

In [2]:
data_csv = pd.read_csv("data/heart.csv")

### Ambil data secara acak untuk prediksi


Kode di bawah bertujuan untuk mengambil satu baris data secara acak dari sebuah DataFrame data_csv dan mengubahnya menjadi sebuah dictionary di mana kunci adalah nama kolom dan nilai adalah nilai dari baris tersebut. 

In [3]:
columns = data_csv.columns.values
rand = random.randint(0, len(data_csv) - 1)

# Mengambil fitur
features = data_csv.values[rand]

# Membuat dictionary input
inputs = {key: value for key, value in zip(columns, features)}
inputs

{'Age': 54,
 'Sex': 'M',
 'ChestPainType': 'ASY',
 'RestingBP': 200,
 'Cholesterol': 198,
 'FastingBS': 0,
 'RestingECG': 'Normal',
 'MaxHR': 142,
 'ExerciseAngina': 'Y',
 'Oldpeak': 2.0,
 'ST_Slope': 'Flat',
 'HeartDisease': 1}

### Membuat helper Function untuk membuat TFRecord
- string_feature(value): Mengonversi string menjadi fitur TFRecord dalam format byte.
- float_feature(value): Mengonversi nilai float menjadi fitur TFRecord dalam format float list.
- int_feature(value): Mengonversi nilai integer menjadi fitur TFRecord dalam format int64 list.
Ketiga fungsi ini adalah helper functions yang memudahkan pembuatan fitur yang sesuai untuk disimpan dalam format TFRecord, yang kemudian dapat digunakan dalam pipeline pelatihan model TensorFlow.

In [4]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

Fungsi def prepare_json di bawah akan mengonversi data dari format dictionary ke format yang diperlukan untuk mengirimkan data ke model TensorFlow Serving melalui HTTP API. Data yang dikirim dalam format JSON ini berisi data yang telah di-encode ke dalam format TFRecord dan dikodekan dalam base64.

In [5]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, str):
            feature_spec[keys] = string_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        else:
            feature_spec[keys] = int_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{"examples": {"b64": base64.b64encode(example).decode()}}]

    return json.dumps(
        {
            "signature_name": "serving_default",
            "instances": result,
        }
    )

### Melakukan prediksi

In [6]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = "https://heart-failure-pred-production.up.railway.app/v1/models/heartpred-model:predict"
    response = requests.post(endpoint, data=json_data, headers={"Content-Type": "application/json"})
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.5:
        return "Pasien tidak mengalami penyakit jantung"
    else:
        return "Pasien mengalami penyakit jantung"

In [7]:
# Menampilkan hasil prediksi
hasil_prediksi = make_predictions(inputs)
print('Fitur Input:', inputs)
print(f'Hasil Prediksi: {hasil_prediksi}')

Fitur Input: {'Age': 54, 'Sex': 'M', 'ChestPainType': 'ASY', 'RestingBP': 200, 'Cholesterol': 198, 'FastingBS': 0, 'RestingECG': 'Normal', 'MaxHR': 142, 'ExerciseAngina': 'Y', 'Oldpeak': 2.0, 'ST_Slope': 'Flat', 'HeartDisease': 1}
Hasil Prediksi: Pasien mengalami penyakit jantung
